In [ ]:
# Utilização da técnica SMOTE para oversampling das imagens

In [1]:
import numpy as np
import pandas as pd
import cv2
import random
import os
import matplotlib.pyplot as plt

In [2]:
path = os.getcwd()+'//..//features2//'

In [ ]:
features = pd.read_csv(path+'features_all.csv', sep=';', index_col=0)

In [14]:
k = 5
N = 200


In [ ]:
def smote(features, k, N, i=1501):
    N = (int)(N/100)
    T = features.shape[0]
    synthetic = pd.DataFrame(data=,index=,columns=)
    
    def normalize_features():
    def post_normalize_features():
    def get_neighbors(sample):
    def populate(neighbors, sample, i):
        for n in N:
            random = random.randint(1,k)
            diff= neighbors.iloc[random] - features.iloc[sample]
            gap = random.uniform(0,1)
            feature_row = features.iloc[sample] + diff*gap
            synthetic_sample = pd.Series(data=feature_row, index=features.columns.values, name=str(i)+'synthetic')
            synthetic.loc[str(i)+'synthetic'] = synthetic_sample
            i=i+1
        
    normalize_features()
    
    for sample in T:
        neighbors = get_neighbors(features.iloc[sample])
        populate(neighbors, sample)
    
    return synthetic

In [ ]:
def treat_target():
    

In [109]:
imgs = sorted(os.listdir(os.getcwd()+'//..//preprocessadas'))

In [117]:
data = pd.DataFrame(data=imgs, columns=['names'])

In [119]:
data.to_csv('img_names.csv', sep=';')

In [133]:
firm1 = pd.read_csv(os.getcwd()+'//..//reference_values//primeiros_dados_palmer_time.csv',sep=';',index_col=0)
firm2 = pd.read_csv(os.getcwd()+'//..//reference_values//ultimos_dados_palmer_time.csv',sep=';',index_col=0)

In [134]:
firm = pd.concat([firm1,firm2])

In [136]:
firm.sort_index().to_csv('time_junto.csv',sep=';')